## Combined Somatic and CHIP variant calling models

In [69]:
library(dplyr)
library(ggplot2)
library(RColorBrewer)
#library(gridExtra)
library(tidyr)
library(caret)
#library(ROCR)
library(pROC)
#library(knitr)

source("~/Desktop/puffin/R/helper_functions.R")
source("~/Desktop/Moffitt_2022/fix_PIDs.R")
options(stringsAsFactors = FALSE)
# set some defaults
options(stringsAsFactors = FALSE)
options(repr.matrix.max.cols=75, repr.matrix.max.rows=100)
formals(table)$useNA <- "always"
formals(write.csv)$row.names <- FALSE
formals(write.csv)$as.is <- TRUE

print(Sys.time())
print(sessionInfo())

[1] "2024-02-01 10:11:46 PST"
R version 4.3.0 (2023-04-21)
Platform: aarch64-apple-darwin20 (64-bit)
Running under: macOS 14.1.1

Matrix products: default
BLAS:   /Library/Frameworks/R.framework/Versions/4.3-arm64/Resources/lib/libRblas.0.dylib 
LAPACK: /Library/Frameworks/R.framework/Versions/4.3-arm64/Resources/lib/libRlapack.dylib;  LAPACK version 3.11.0

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

time zone: America/Los_Angeles
tzcode source: internal

attached base packages:
[1] grid      stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] ComplexHeatmap_2.16.0 maftools_2.16.0       eulerr_7.0.0         
 [4] glue_1.6.2            stringr_1.5.0         pROC_1.18.4          
 [7] caret_6.0-94          lattice_0.21-8        tidyr_1.3.0          
[10] RColorBrewer_1.1-3    ggplot2_3.4.2         dplyr_1.1.2          

loaded via a namespace (and not attached):
 [1] tidyselect_1.2.0     timeDate_4022.10

In [70]:
# Moffitt test data sets
all.snv.plasma <- read.csv("../Moffitt_2022/MIBC/data/ATLAS/PRDC-MOFFITT-MIBC-22002/07_27_23_update/PRDC-MOFFITT-MIBC-22002-ATLAS_Variant_all_07-27-2023_2023-07-27_plasma_marked.csv")

all.snv.plasma = all.snv.plasma %>% mutate(PatientID.old = PatientID,
                                           PatientID = fix_PIDs(PatientID.old))
table(all.snv.plasma$VariantType.old)
table(all.snv.plasma$VariantType)
nrow(all.snv.plasma)
table(all.snv.plasma$SpecimenType)
all.snv.plasma %>% group_by(PatientID) %>% summarize(n_mutations=n()) 

### Matched normal pbmc 
all.pbmc = read.csv("../CHIP/PRDC-MOFFITT-MIBC-22002-WES_Variant_all_2022-12-28_pbmc.csv", as.is=T)
table(all.pbmc$SpecimenType)
# fix old patient id numbers in this file
all.pbmc = all.pbmc %>% mutate(PatientID.old = PatientID,
                               PatientID = fix_PIDs(PatientID.old))
# any patients without germline data?
stopifnot(all(unique(all.snv.plasma$PatientID) %in% unique(all.pbmc$PatientID)))
dim(all.snv.plasma)
head(all.snv.plasma)
# Add patient ID to VariantID for more unique ID
all.snv.plasma <- all.snv.plasma %>% mutate(VariantID.2=paste0(PatientID, ":", VariantID))
length(unique(all.snv.plasma$VariantID.2)) # make sure each variants get unique ID. matches number of rows (n=55644)


# Read in tumor only ffpe
#all.snv.tumor <- read.csv("data/PRDC-MOFFITT-MIBC-22002-WES_Variant_all_2022-12-28_ffpe_marked.csv", as.is=T)
#all.snv.tumor = all.snv.tumor %>% mutate(PatientID.old = PatientID,
#                                         PatientID = fix_PIDs(PatientID.old))
#all.snv.tumor %>% group_by(PatientID) %>% summarize(n_mutations=n())

# any patients without tumor data?
# stopifnot(all(unique(all.snv.plasma$PatientID) %in% unique(all.snv.tumor$PatientID))) 

# some patients don't have tumor data, filter these out
#all.snv.plasma = all.snv.plasma %>% filter(PatientID %in% unique(all.snv.tumor$PatientID))
#dim(all.snv.plasma) #should be 48097 total variants


### Whether variant exists in public databases = binary classification
all.snv.plasma <- all.snv.plasma %>% mutate(in.dbSNP=ifelse(is.na(dbSNP), as.character(0), "in.dbSNP"),
                                            in.genome1000=ifelse(genome1000=="", as.character(0), "in.genome1000"),
                                            in.cosmic=ifelse(is.na(COSMIC), as.character(0), "in.cosmic"))

 [1] "173736" "172592" "175702" "89200"  "174438" "172956" "174280" "172109"
 [9] "170762" "170594" "172163" "173129" "171660" "175253" "171351" "175743"
[17] "172422" "171126" "173657" "174688" "175827" "173149" "173983"



 CHIP mutations        Germline Likely germline  Likely somatic         Somatic 
             56           47018            1928            6629              13 
           <NA> 
              0 


       CHIP mutations              Germline     Likely background 
                   33                 50450                  1216 
Likely CHIP mutations       Likely germline        Likely somatic 
                   36                  1101                   600 
              Somatic                  <NA> 
                 2208                     0 

[1] 55644


Plasma   <NA> 
 55644      0 

...


Buffy Coat       <NA> 
    591299          0 

 [1] "173983" "170594" "172163" "172401" "172592" "172422" "172403" "173149"
 [9] "174438" "174688" "170762" "171351" "172109" "172956" "173129" "171660"
[17] "173736" "174280" "173657" "175253" "89200"  "175702" "175743" "171126"
[25] "175827"


[1] 55644   101

,X,seqnames,start,end,width,strand,ref,alt,totalDepth,refDepth,altDepth,sampleNames,VariantFreq,SYMBOL,GeneID,Feature,HGVSc,HGVSp,Amino_acids,Codons,BIOTYPE,EXON,INTRON,Consequence,DISTANCE,Existing_variation,IMPACT,VARIANT_CLASS,CLIN_SIG,Clinvar,dbSNP,COSMIC,COSMIC.CNT,genome1000,AF,CANONICAL,filteredCnt,DSCnt,⋯,CopyNumber,zScore.cnv,coVariant,ol.coVariant,snvmember,mergedVariant,coVariantFilter,sideVariant,sideVariant.AF.ratio,prevalenceInternal,filterPrevalence,filterScore,finalKeep,VariantID,sampleDir,VariantType,depth,FilterType,sampleFolder,PatientID,externalSampleID,trialVisitNum,SpecimenType,ID,highFrequent.inbatch,concordant,PatientID.old,totalDepth.Baseline,dscnt.Baseline.filtered,altDepth.Baseline,altDepth.Baseline.filtered,AF.Baseline,odds.ratio,pvalue,MAF.diff,VariantType.old,finalKeep.old
,<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<int>,⋯,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>,<dbl>,<dbl>,<lgl>,<dbl>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<lgl>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,1,chr1,11187893,11187893,1,*,T,C,3769,4,3765,P224641,99.89387,MTOR,2475,NM_004958.4,c.6034-30A>G,p.(=),NA,NA,protein_coding,NA,'43/57',intron_variant,NA,rs1770344,MODIFIER,SNV,NA,NA,NA,NA,NA,rs1770344,1.0000,yes,2991,2500,⋯,2.28,0.184,,,NA,,TRUE,,NA,0.0584,TRUE,8.475,FALSE,chr1:11187893:T:C,/prednet/bdata21/OutputByRunB21/221123_A00934_0231_BHLL7KDSX5_runtask_221126000739_c84c4879/dsrun1.7.1/work/f5/fd5aef3e8a495f2a82f720c5c885d3,Germline,high,otherFinal,P224641_221123,173736,06S22025850,Pre Op,Plasma,P224641:chr1:11187893:T:C,23,FALSE,173736,922,28,922,736,100.0,0.9989388,0.5185111,-0.10612895,Germline,TRUE
2,2,chr1,11205058,11205058,1,*,C,T,3506,26,3480,P224641,99.25841,MTOR,2475,NM_004958.4,c.4731G>A,p.Ala1577=,p.A1577A,gcG/gcA,protein_coding,'33/58',NA,synonymous_variant,NA,rs1057079&COSV63875443,LOW,SNV,benign,Benign:1,rs1057079,COSM4142146,3,rs1057079,0.8165,yes,3001,1943,⋯,2.12,0.265,,,NA,,TRUE,,NA,0.9180,TRUE,9.800,FALSE,chr1:11205058:C:T,/prednet/bdata21/OutputByRunB21/221123_A00934_0231_BHLL7KDSX5_runtask_221126000739_c84c4879/dsrun1.7.1/work/f5/fd5aef3e8a495f2a82f720c5c885d3,Germline,high,otherFinal,P224641_221123,173736,06S22025850,Pre Op,Plasma,P224641:chr1:11205058:C:T,21,FALSE,173736,512,13,508,408,99.2,1.0004027,0.5110042,0.05841415,Germline,TRUE
3,3,chr1,11288758,11288758,1,*,G,A,3818,27,3791,P224641,99.29282,MTOR,2475,NM_004958.4,c.2997C>T,p.Asn999=,p.N999N,aaC/aaT,protein_coding,'19/58',NA,synonymous_variant,NA,rs1064261&COSV63873449,LOW,SNV,NA,NA,NA,COSM4142152,2,rs1064261,0.9191,yes,3171,2524,⋯,2.30,0.846,chr1:11288618:G14G124G:A14C124A;chr1:11288633:G124G:C124A;chr1:11288618:G139G:A139A,0.17;0.00422;0.000315,NA,,TRUE,,NA,0.9290,TRUE,9.050,FALSE,chr1:11288758:G:A,/prednet/bdata21/OutputByRunB21/221123_A00934_0231_BHLL7KDSX5_runtask_221126000739_c84c4879/dsrun1.7.1/work/f5/fd5aef3e8a495f2a82f720c5c885d3,Germline,high,otherFinal,P224641_221123,173736,06S22025850,Pre Op,Plasma,P224641:chr1:11288758:G:A,20,FALSE,173736,1013,31,1008,844,99.5,0.9978536,0.5270965,-0.20717653,Germline,TRUE
4,4,chr1,11301714,11301714,1,*,A,G,2724,7,2717,P224641,99.74302,MTOR,2475,NM_004958.4,c.1437T>C,p.Asp479=,p.D479D,gaT/gaC,protein_coding,'10/58',NA,synonymous_variant,NA,rs1135172&COSV63873456,LOW,SNV,benign,Benign:1,rs1135172,COSM4142157,2,rs1135172,0.8789,yes,1917,1688,⋯,2.07,1.050,,,NA,,TRUE,,NA,0.9360,TRUE,8.975,FALSE,chr1:11301714:A:G,/prednet/bdata21/OutputByRunB21/221123_A00934_0231_BHLL7KDSX5_runtask_221126000739_c84c4879/dsrun1.7.1/work/f5/fd5aef3e8a495f2a82f720c5c885d3,Germline,high,otherFinal,P224641_221123,173736,06S22025850,Pre Op,Plasma,P224641:chr1:11301714:A:G,20,FALSE,173736,752,16,752,648,100.0,0.9974305,0.5292151,-0.25697504,Germline,TRUE
5,5,chr1,114224782,11422478

[1] 55644

In [ ]:
## implement Tram's best somatic calling model using predicine raw call as a feature and checkGerm paired as truth
## improve Tram's model?
## basic stack Trams model and Kevin's model stepwise, threshold diagram
## combine both models into one big logistic model
# other model architectures, boosting, feature importance
## compare agaisn raw calls, Mutect2, random forest

## Tram's model

In [71]:
# Add features

# Across ALL patient samples at each variant locus. Find the max VAF across the dataset (t_maj)
all.snv.plasma.f <- as.data.frame(all.snv.plasma %>% group_by(VariantID) %>% mutate(max.VaraintFreq.group = max(VariantFreq)))
unique(all.snv.plasma.f$max.VaraintFreq.group[all.snv.plasma.f$seqnames == "chr16" & all.snv.plasma.f$start == "136888"]) 
# Check that for the same position across patients, should have the same t_maj


########## 3. t_alt_freq: ##########
# The fraction of reads corresponding to the alternate allele
# I think this is just VAF (VariantFreq) which is always the alt allele


########## 4. max_cosmic_count:	##########
# The number of times this allele has been observed in the COSMIC
# Use the COSMIC.CNT column

# First, we need to change the COSMIC.CNT column to integer (it is now character)
all.snv.plasma.f$COSMIC.CNT <- as.numeric(all.snv.plasma.f$COSMIC.CNT)
#class(all.snv.plasma$COSMIC.CNT)

all.snv.plasma.f <- all.snv.plasma.f %>% group_by(VariantID) %>% # added alt here since sometimes multiple alleles at a site
  mutate(max_COSMIC_CNT.group = ifelse(!(all(is.na(COSMIC.CNT))), max(COSMIC.CNT, na.rm = TRUE), 0))
# If not in COSMIC = NA = set to zero

[1] 99.93683

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [72]:
########## 5. pop_max: ########## 
# The maximum population frequency of the allele across multiple germline databases
## CHECK dbNSNP?
## https://www.internationalgenome.org/faq/how-do-i-find-out-information-about-a-single-variant/


########################### ADD VEP POP ALLELE FREQS  #######################

# Instead of PatientID we can use SampleName as patient identifier because VEP only includes sample ID and no patient.
all.snv.plasma.f = all.snv.plasma.f %>% 
  mutate(VEP.ID =paste0(sampleNames, ":", VariantID)) 

# Load VEP and create VEP.ID to match atlas variants
vep <- read.delim("./Tram_working_files/NMIBC_plasma_ATLAS_variant_all_marked_dist2end_sel_wVEP_09272023.tsv", 
                  sep = ",",header = TRUE,as.is = T)
vep <- vep %>% 
  mutate(VEP.ID =paste0(vep$sampleNames, ":", VariantID)) 
table(vep$sampleNames)
table(all.snv.plasma.f$sampleNames)

#length(intersect(unique(vep$sampleNames), unique(all.snv.plasma$sampleNames))) # how many patients are in common for both (n=20)
#length(intersect(all.snv.plasma$VEP.ID, vep$VEP.ID)) # How many variants in common = 48097

# Grab columns we want from VEP
sort(names(vep))
# AF = Frequency of existing variant in 1000 Genomes
# gnomADe_AF - Frequency of existing variant in gnomAD exomes combined population -- (Not available for our data)
# MAX_AF - Maximum observed allele frequency in 1000 Genomes, ESP and gnomAD
# MAX_AF_POPS - Populations in which maximum allele frequency was observed
# SOMATIC - Somatic status of existing variant(s); multiple values correspond to multiple values in the Existing_variation field
# PHENO - Indicates if existing variant is associated with a phenotype, disease or trait; multiple values correspond to multiple values in the Existing_variation field
# IMPACT - consequence of variant

# NOTE: missing ASN_AF population in this output?? Wonder why...might want to check for future
# AF.diff, AF.th, AF already in the ATLAS database, don't need to include
sel_col <- c("VEP.ID", "AFR_AF", "AMR_AF" ,"EUR_AF", "EAS_AF" ,"SAS_AF", "MAX_AF", "SOMATIC", "PHENO", "mapQ_mean")
vep.sel = vep %>% select(sel_col)
head(vep.sel)

# join VEP columns
all.snv.plasma.f <- all.snv.plasma.f %>% left_join(vep.sel, by="VEP.ID") %>% 
        mutate("mapQ_mean" = as.numeric(mapQ_mean))


########## 6. snp_vaf_bin_00: ##########
# The number of informative SNPs in the local copy number neighborhood with VAF between 0 and 0.05

# Need to get copy number neighborhoods
# Filter for VAF bin
# Count how many SNPs are there
# germline variant databases and copy-number segments, we identify neighboring heterozygous germline SNPs of similar copy number, and create a histogram of variant counts with 20 non- overlapping VAF bins 
# the copy number for each variant is represented by features derived from copy-number segmentation data and variant calls. Briefly, using germline variant databases and copy-number segments, we identify neighboring heterozygous germline SNPs of similar copy number, and create a histogram of variant counts with 20 non- overlapping VAF bins (see “Methods”).

# possible features to use
# CopyNumber column
# average CopyNumber for this snv and other snvs within ~10 MB (?could use average human genome LD MB value here) for the patient

# local 1MB bin CN estimate and/or coverage zscore from additional LP-WGS data


########## 7. Somatic Count: ##########
# Count total number of somatic variants per patient
nrow(all.snv.plasma.f)
df.counts <- all.snv.plasma.f %>% filter(grepl('somatic|chip', VariantType.old, ignore.case=T)) %>% 
    group_by(PatientID) %>%  summarize("allSomaticCount" = n())
all.snv.plasma.f <- all.snv.plasma.f %>% left_join(df.counts %>% select(PatientID, allSomaticCount), by="PatientID")
nrow(all.snv.plasma.f)

## RATIONALE FROM THE MCLAUHLIN PAPER: 
# "Interestingly, in XGBoost and LightGBM, the most important feature is count 
#(the total number of variants to classify in the sample) rather than pop_max, which appears as third most important. 
# The lower dependency on population databases likely underlies the elimination of racially biased TMB inflation in these"models. # Or perhaps knowing count, the total number of mutations to classify—which depends largely on the number of rare germline variants absent from the biased databases—allows LightGBM and XGBoost to recognize and make better decisions with samples from patients in underrepresented groups."

# Check that each patient has unique total Variant count
all.snv.plasma.f %>%  group_by(PatientID) %>% summarise(count = unique(allSomaticCount))


P217576 P224597 P224601 P224606 P224608 P224615 P224619 P224623 P224624 P224625 
   2581    2634    2467    2337    2379    2296    2398    2552    2448    2515 
P224628 P224630 P224633 P224637 P224641 P224642 P224644 P224646 P224648 P224651 
   2359    2461    2332    2478    2438    2405    2360    2402    2427    2401 
P224652 P224653 P224655    <NA> 
   2410    2288    2276       0 


P217576 P224597 P224601 P224606 P224608 P224615 P224619 P224623 P224624 P224625 
   2581    2634    2467    2337    2379    2296    2398    2552    2448    2515 
P224628 P224630 P224633 P224637 P224641 P224642 P224644 P224646 P224648 P224651 
   2359    2461    2332    2478    2438    2405    2360    2402    2427    2401 
P224652 P224653 P224655    <NA> 
   2410    2288    2276       0 

[1] "AF"                           "AF.diff"                     
  [3] "AF.th"                        "AFR_AF"                      
  [5] "Allele"                       "alt"                         
  [7] "altDepth"                     "altLOD"                      
  [9] "Amino_acids"                  "AMR_AF"                      
 [11] "avgEntropy"                   "BAM_EDIT"                    
 [13] "bgFilter"                     "BIOTYPE"                     
 [15] "CANONICAL"                    "cDNA_position"               
 [17] "CDS_position"                 "CLIN_SIG"                    
 [19] "Clinvar"                      "Codons"                      
 [21] "Consequence"                  "CopyNumber"                  
 [23] "COSMIC"                       "COSMIC.CNT"                  
 [25] "coVariant"                    "coVariantFilter"             
 [27] "dbSNP"                        "DISTANCE"                    
 [29] "DS.filter"                    "DSCnt"                       
 [31] "EAS_AF"                       "end"                         
 [33] "entropy"                      "EUR_AF"                      
 [35] "Existing_variation"           "EXON"                        
 [37] "fc.ds.ratio"                  "Feature"                     
 [39] "Feature_type"                 "filtered_tlen_mean"          
 [41] "filtered_tlen_N"              "filtered_tlen_std"           
 [43] "filtered_toend_numunique"     "filtered_total_base_count"   
 [45] "filtered.consensus"           "filteredCnt"                 
 [47] "filteredDSCnt"                "filterPrevalence"            
 [49] "filterScore"                  "FilterType"                  
 [51] "filterVarType"                "finalKeep"                   
 [53] "FLAGS"                        "GC.Percent"                  
 [55] "Gene"                         "GENE_PHENO"                  
 [57] "GeneID"                       "generalFilter"               
 [59] "genome1000"                   "GIVEN_REF"                   
 [61] "heteroLOD"                    "HGNC_ID"                     
 [63] "HGVS_OFFSET"                  "HGVSc"                       
 [65] "HGVSp"                        "highFrequent"                
 [67] "highQ.ratio"                  "homoLOD"                     
 [69] "ID"                           "IMPACT"                      
 [71] "importantVariant"             "INTRON"                      
 [73] "KeepStatus"                   "localVariantFilter"          
 [75] "mapQ_mean"                    "mapQ_std"                    
 [77] "MAX_AF"                       "MAX_AF_POPS"                 
 [79] "mergedVariant"                "MNV"                         
 [81] "ol.coVariant"                 "onTarget"                    
 [83] "partial_strand_cnt"           "partial_strand_mean"         
 [85] "partial_strand_std"           "PHENO"                       
 [87] "prevalenceInternal"           "Protein_position"            
 [89] "pseudoDSCnt"                  "Qscore"                      
 [91] "ratio.fragLen"                "ReadEndFilter"               
 [93] "ref"                          "ref.avgEntropy"              
 [95] "ref.DSCnt"                    "ref.entropy"                 
 [97] "ref.filtered_tlen_mean"       "ref.filtered_tlen_N"         
 [99] "ref.filtered_tlen_std"        "ref.filtered_toend_numunique"
[101] "ref.filteredCnt"              "ref.filteredDSCnt"           
[103] "ref.mapQ_mean"                "ref.score_mean"              
[105] "ref.strand_both_cnt"          "ref.strand_forward_cnt"      
[107] "ref.strand_reverse_cnt"       "ref.toend_mean"              
[109] "ref.toend_std"                "ref.tomend_mean"             
[111] "ref.tomend_median"            "ref.tomend_numunique"        
[113] "ref.tomend_std"               "ref.width"                   
[115] "refDepth"                     "REFSEQ_MATCH"                
[117] "REFSEQ_OFFSET"                "repeatRegion"       

,VEP.ID,AFR_AF,AMR_AF,EUR_AF,EAS_AF,SAS_AF,MAX_AF,SOMATIC,PHENO,mapQ_mean
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
1,P224641:chr1:3644349:A:G,0.7753,0.6758,0.825,0.3353,0.6063,0.8250,NA,NA,59.9787
2,P224641:chr1:3644374:A:G,0.7723,0.6758,0.828,0.3353,0.6074,0.8280,NA,NA,59.9843
3,P224641:chr1:9770690:C:CAG,NA,NA,NA,NA,NA,0.4378,NA,NA,59.9154
4,P224641:chr1:9775888:A:C,NA,NA,NA,NA,NA,NA,NA,NA,58.4577
5,P224641:chr1:9775898:A:C,NA,NA,NA,NA,NA,NA,NA,NA,59.2995
6,P224641:chr1:9775903:G:C,NA,NA,NA,NA,NA,NA,NA,NA,58.5718


[1] 55644

[1] 55644

PatientID,count
<chr>,<int>
170594,307
170762,333
171126,304
171351,280
171660,325
172109,239
172163,273
172422,261
172592,239


In [73]:
########## 8. Ontology: ##########
# Categorical Coding mutation subclassification. McLaughlin's categories: inframe_indel, missense, nonsense, frame_shift_indel
# Looks like we can use the "Consequence" column of the data. Maybe in the future we bin these?

# here is a previously designed binning function for this. you can also create your own reclassification and 
#insert as var.reduc.set: 
source("~/Desktop/puffin/R/concordance_oncoprint.R")
all.snv.plasma.f$Consequence.short <- recode.variants(all.snv.plasma.f$Consequence, var.reduc.set = "consequence_reduced")
dim(all.snv.plasma.f)
names(all.snv.plasma.f)[100:length(all.snv.plasma.f)] #Check new features added

[1] "Original annotations:"
annotations
                                    3_prime_UTR_variant 
                                                   1299 
                                    5_prime_UTR_variant 
                                                    586 
                                coding_sequence_variant 
                                                     26 
                 coding_sequence_variant&intron_variant 
                                                      1 
                                downstream_gene_variant 
                                                    415 
                                     frameshift_variant 
                                                     88 
               frameshift_variant&splice_region_variant 
                                                      2 
frameshift_variant&splice_region_variant&intron_variant 
                                                    115 
                                       inframe_d

[1] 55644   119

[1] "VariantType.old"       "finalKeep.old"         "VariantID.2"          
 [4] "in.dbSNP"              "in.genome1000"         "in.cosmic"            
 [7] "max.VaraintFreq.group" "max_COSMIC_CNT.group"  "VEP.ID"               
[10] "AFR_AF"                "AMR_AF"                "EUR_AF"               
[13] "EAS_AF"                "SAS_AF"                "MAX_AF"               
[16] "SOMATIC"               "PHENO"                 "mapQ_mean"            
[19] "allSomaticCount"       "Consequence.short"

In [74]:
########## COSMIC MUTATION SBS SIGNATURE: ##########

########## 9. trinucleotide_context: ##########
# The three surrounding nucleotides (1 bp upstream, the locus itself, and 1 bp downstream) in the reference genome for the SNV (non_SBS = non-single base-pair substitution, i.e, indels)

######### From mutationSignature workflow:
# We will add the tri "context" column
# R script: "get-trinucleotide.R"

# Load in our trinucleotide context and mutation motifs
mut <- read.delim("./Tram_working_files/tri-mut-context_SEP.txt", sep = "\t", header = T, as.is=T)
dim(mut)

mut <- mut %>% mutate(VariantID.2=paste0(PatientID, ":", VariantID))
mut <- mut %>%  select(VariantID.2, context, mutType, mutMotif) # Just grab relevant info to join new columns
mut$VariantID.2 <- as.character(mut$VariantID.2)
head(mut)

# Merge tri and mut context to ALL PLASMA variants
### REMOVE INDELS
all.snv.plasma.f = all.snv.plasma.f %>% filter(width<2)
dim(all.snv.plasma.f)
all.snv.plasma.f <- all.snv.plasma.f %>% left_join(mut, by="VariantID.2")

names(all.snv.plasma.f)[100:length(all.snv.plasma.f)] # Should see the tri-nucle, mutation Type, and signature columns added

########## substitution_type: ##########	
# Which of the 6 unique transition and transversion types characterizes the single-base-substitution (non-SBS for indels)
# C>G, C>T, T>A, T>C, T>G, non-SBS, C>A

#mut <- read.delim("tri-mut-context.txt", sep = "\t", header = T, as.is=T)
# Merge on the new trinucleotide/motif columns. NA for if not available for variants.
#all.snv.plasma <- all.snv.plasma  %>% left_join(mut, by="ID", suffix = c("", ".drop")) %>%
#  select(-ends_with(".drop"))
#head(all.snv.plasma)

## For variants with mutType/mutMotif (indels) = set NA to nonSBS
# all.snv.plasma = all.snv.plasma %>% mutate(mutType=ifelse(nchar(ref)>1 | nchar(alt)>1, "nonSBS", mutType))
#all.snv.plasma$mutMotif[is.na(all.snv.plasma$mutMotif)] <- "nonSBS"

# NOTE: MAYBE LEAVE THIS NA?
table(all.snv.plasma.f$mutType)
table(all.snv.plasma.f$mutMotif)

dim(all.snv.plasma.f)

[1] 49912     9

,VariantID.2,context,mutType,mutMotif
,<chr>,<chr>,<chr>,<chr>
1,173736:chr1:11187893:T:C,ATC,A[T>C]C,T>C
2,173736:chr1:11205058:C:T,TCG,T[C>T]G,C>T
3,173736:chr1:11288758:G:A,CGT,A[C>T]G,C>T
4,173736:chr1:11301714:A:G,CAT,A[T>C]G,T>C
5,173736:chr1:117690272:G:A,CGA,T[C>T]G,C>T
6,173736:chr1:118165577:C:G,TCA,T[C>G]A,C>G


[1] 52128   119

[1] "VariantType.old"       "finalKeep.old"         "VariantID.2"          
 [4] "in.dbSNP"              "in.genome1000"         "in.cosmic"            
 [7] "max.VaraintFreq.group" "max_COSMIC_CNT.group"  "VEP.ID"               
[10] "AFR_AF"                "AMR_AF"                "EUR_AF"               
[13] "EAS_AF"                "SAS_AF"                "MAX_AF"               
[16] "SOMATIC"               "PHENO"                 "mapQ_mean"            
[19] "allSomaticCount"       "Consequence.short"     "context"              
[22] "mutType"               "mutMotif"


A[C>A]A A[C>A]C A[C>A]G A[C>A]T A[C>G]A A[C>G]C A[C>G]G A[C>G]T A[C>T]A A[C>T]C 
    400     252     201     185     541     231     277     342    1201     779 
A[C>T]G A[C>T]T A[T>A]A A[T>A]C A[T>A]G A[T>A]T A[T>C]A A[T>C]C A[T>C]G A[T>C]T 
   2324    1095     148     288     252     163    1097     552    2325    1067 
A[T>G]A A[T>G]C A[T>G]G A[T>G]T C[C>A]A C[C>A]C C[C>A]G C[C>A]T C[C>G]A C[C>G]C 
    189     171     172     126     277     278     276     199     248     331 
C[C>G]G C[C>G]T C[C>T]A C[C>T]C C[C>T]G C[C>T]T C[T>A]A C[T>A]C C[T>A]G C[T>A]T 
    563     410     645    1105    2261    1027     102     321     235     309 
C[T>C]A C[T>C]C C[T>C]G C[T>C]T C[T>G]A C[T>G]C C[T>G]G C[T>G]T G[C>A]A G[C>A]C 
    637     801    2118     740     111     361     505     264     327     326 
G[C>A]G G[C>A]T G[C>G]A G[C>G]C G[C>G]G G[C>G]T G[C>T]A G[C>T]C G[C>T]G G[C>T]T 
    154     111     372     288     618     276     615     827    1654     681 
G[T>A]A G[T>A]C G[T>A]G G[T


  C>A   C>G   C>T   T>A   T>C   T>G  <NA> 
 3713  5981 17585  2941 15502  4190  2216 

[1] 52128   122

In [ ]:
### FROM SIGPROFILERASSIGNMENT_R PROGRAM : 
### MORE DETAILED CODE TO GENERATE THESE OUTPUTS IN: "SigProfileExtractor-workflow.R" script. Uploaded into the test_data OneDrive
#https://github.com/AlexandrovLab/SigProfilerAssignment
#The script to match the program's outputs with the variants is pretty rough and I limited to only single-base substitutions but I think the code could definitely use another look through


# Read in outputs -- which are currently separate for each patient
mydir = ("./Tram_working_files/Assignment_Solution/Activities/Decomposed_Mutation_Probabilities")
myfiles = list.files(path=mydir, pattern = ".txt", full.names = T)
mytitles = basename(myfiles)
mytitles
names <- sub("Decomposed_Mutation_Probabilities_", "", mytitles)
names <- sub(".txt", "", names)
names <- sub("P", "", names) # remove the P from the patentID to match plasma

tables <- lapply(myfiles, read.delim, as.is=T, sep="\t", header=T)
names(tables) <- names

# Load in the plasma data - lets do a test df in case it messes up
test.plasma <- all.snv.plasma.f

#create empty list with length of zero
sig_list <- list()

#create empty list of length 10
sig_list <- vector(mode='list', length=23)
names(sig_list) <- names


# Grab a list of PatientIDs. Loop through each one and grab the SBS probabilities.
# This is a bit complicated because we cannot get VariantID.2 from just the outputs. We need to compare it to the VCF files that were provided as input, and then take the ref and alt alleles from those to create a VariantID.2 column that will match plasma. 

for (i in 1:length(names)){
   
   c.patient <- names[i]
   
   c.df <- tables[[c.patient]]
   
   # Remove the P from the sampleName
   c.df$Sample.Names <- gsub("P", "", c.df$Sample.Names)
   
   # Load in the VCF coords inputs for the SigProfiler
   vcfname <- paste0("./Tram_working_files//Documents/Code/sigExtractor/atlas_vcfs/input/", "P", c.patient, ".vcf")
   vcf <- read.delim(vcfname, as.is=T, sep = "\t", header = F)
   
   nrow(c.df)
   nrow(vcf) # Usually more VCF variants because these still include indels
   
   
   # Limit VCF to only SBS -- remove indels from VCF coords 
   vcf_SBS <- vcf %>%
     filter(nchar(V4) == 1 & nchar(V5) == 1)
   
   # In the VCF, sometimes there are positions that have two calls... remove those rows
   vcf_SBS <- vcf_SBS %>%
     distinct(V1, V2, .keep_all = TRUE)
   
   # Check set intersect again
   c.df <- c.df %>% mutate(VariantID.2=paste0(Sample.Names,":", "chr", Chr, ":", Pos))
   vcf <- vcf_SBS %>% mutate(VariantID.2=paste0(V3,":", "chr", V1, ":", V2))
   
   # There will be more c.df now because of the two-call positions
   nrow(c.df)
   nrow(vcf)
   
   vcf$VariantID.2[which(duplicated(vcf$VariantID.2))] # should be none becuase we removed the duplicated coords
   dups <- c.df$VariantID.2[which(duplicated(c.df$VariantID.2))]
   dups
   
   # remove the duplicates 2bp entries from SigProfiler
   c.df <- c.df %>% filter(!c.df$VariantID.2 %in% dups)
   
   nrow(c.df)
   nrow(vcf)
   
   # Just grab the necessary mutSigs from the unique variants in VCF coordinates
   merged_df <- merge(c.df, vcf, by = "VariantID.2", all = FALSE) # only what's in common
   
   # Replace the Variant2 column ID to match plasma dataset format (tag on ref and alt allele)
   matched_rows <- merged_df %>% mutate(VariantID.2=paste0(VariantID.2,":", V4, ":", V5))
   
   # Add SigProfile info to the plasma dataset
   matched_rows <- matched_rows %>% select(starts_with("SBS"), "VariantID.2", "MutationType") # Grab necessary columns
   
   #Check how many variants have SigProfile info for
   length(intersect(matched_rows$VariantID.2, test.plasma$VariantID.2))
   
   
   # add the new data set to the sig_list
   
   sig_list[[c.patient]] <- matched_rows

}

# Combine into one dataframe
c.df <- bind_rows(sig_list)

# Remove any SBS signatures columns without any probabilities assigned
c.df <- c.df[, !apply(c.df, 2, function(x) all(x == 0))]
column_names <- colnames(c.df)
column_names[grepl("^SBS", column_names)] # 10 signatures

# For SBS signatures without probs, set to zero
c.df[is.na(c.df)] <- 0

# left_join the SBS signatures to our plasma data
test.plasma <- test.plasma %>% left_join(c.df, by="VariantID.2")

dim(test.plasma) # 133 columns
names(test.plasma)[100:length(test.plasma)] # Should see all the SBS signature columns now.
head(test.plasma[,110:length(test.plasma)])

#all.snv.plasma <- test.plasma

In [ ]:
all.snv.plasma <- test.plasma

In [75]:
######################################################
########## ADD PATIENT LEVEL RELEVANT INFO: ##########
######################################################

#### NUMBER OF COSMIC SIGNATURES PER INDIVIDUAL ####

SigPat <- read.delim("./Tram_working_files/Assignment_Solution/Activities/Assignment_Solution_Activities.txt", as.is=T, header = T, sep="\t")

# remove the patient column
SigPat$Samples <- gsub("P", "", SigPat$Samples)

SigPat <- as.data.frame(SigPat)

# Get column names with counts greater than zero
SBS_names <- apply(SigPat[, -1] > 0, 1, function(row) {
  colnames(SigPat)[-1][row]
})


SBS_count <- apply(SigPat[, -1] > 0, 1, sum)
SBS_count <- as.data.frame(cbind(unique(SigPat$Samples), SBS_count))

colnames(SBS_count) <- c("PatientID", "SBS_Count")
SBS_count$SBS_Count <- as.integer(SBS_count$SBS_Count)

# Left_join number of mut sigs to all plasma data
all.snv.plasma <- all.snv.plasma %>% left_join(SBS_count, by="PatientID")
names(all.snv.plasma)[100:length(all.snv.plasma)]


#### PATIENT CLINICAL DATA ####
NGSQC <- read.csv("~/Desktop/Moffitt_2022/MIBC/data/ATLAS/PRDC-MOFFITT-MIBC-22002/07_27_23_update/PRDC-MOFFITT-MIBC-22002-ATLAS_NGSQC_plasma_updated_2023-07-28.csv", header = T)
NGSQC = NGSQC %>% mutate(PatientID.old = PatientID,
                                         PatientID = fix_PIDs(PatientID.old))

# TMB
patient_cols <- c("PatientID", "pTMB_norm.adj", "pTMB_norm.weighted", "Fragment_Size_Bandwidth", "Fragment_Size_Mode", "maxAF_TMB", "tumorFraction")

NGSQC <- NGSQC %>% select(all_of(patient_cols))
all.snv.plasma <- all.snv.plasma %>% left_join(NGSQC, by="PatientID")
names(all.snv.plasma)[100:length(all.snv.plasma)]

## Save new plasma variants 
#all.snv.plasma <- as.data.frame(all.snv.plasma)
#save(all.snv.plasma, file = "McL_plasma.RData")
#load("McL_plasma.RData")


#############################################################################
################### COMPLETED ADDING MCLAUGHLIN FEATURES  ###################
#############################################################################

[1] "VariantType.old" "finalKeep.old"   "VariantID.2"     "in.dbSNP"       
[5] "in.genome1000"   "in.cosmic"       "SBS_Count"

 [1] "173736" "172592" "175702" "89200"  "174438" "172956" "174280" "172109"
 [9] "170762" "170594" "172163" "173129" "171660" "175253" "171351" "175743"
[17] "172422" "171126" "173657" "174688" "175827" "173149" "173983"


[1] "VariantType.old"         "finalKeep.old"          
 [3] "VariantID.2"             "in.dbSNP"               
 [5] "in.genome1000"           "in.cosmic"              
 [7] "SBS_Count"               "pTMB_norm.adj"          
 [9] "pTMB_norm.weighted"      "Fragment_Size_Bandwidth"
[11] "Fragment_Size_Mode"      "maxAF_TMB"              
[13] "tumorFraction"

In [78]:
# create variant identifier columns
# In the CHIP data, it made sense to check within just somatic calls -- but for us, we want to check ALL plasma variants
all.snv.plasma.McL = all.snv.plasma %>% 
  mutate(VariantID.2=paste0(PatientID, ":", VariantID),
         Plasma.VariantFreq = VariantFreq) 

# these features can correlate with truth labels, but will not be used for training as they will not be available for
# new/holdout data
all.pbmc_ = all.pbmc %>% mutate(VariantID.2=paste0(PatientID, ":", VariantID), Germ.VariantFreq = VariantFreq,
                                Germ.VariantType = VariantType) %>% # might need this info, mostly Germline, some might be CHIP
  select(VariantID.2, Germ.VariantFreq, Germ.VariantType)

# for variants with Germ.VariantFreq values that are NA (missing from germline sample), set Germ.VariantFreq to 0.0%
all.snv.plasma.McL = all.snv.plasma.McL %>% left_join(all.pbmc_, by="VariantID.2") %>%
  mutate(Germ.VariantFreq=ifelse(is.na(Germ.VariantFreq), 0, Germ.VariantFreq),
         in.pbmc = factor(Germ.VariantFreq>0)) # Create another column for updated germline freq for somatic calls 

# all.snv.tumor_ = all.snv.tumor %>% 
#   mutate(VariantID.2=paste0(PatientID, ":", VariantID), Tumor.VariantFreq = VariantFreq, Tumor.VariantType = VariantType) %>%
#   select(VariantID.2, Tumor.VariantFreq, Tumor.VariantType) # Add the tumor variant type and freq
# all.snv.plasma.McL = all.snv.plasma.McL %>% left_join(all.snv.tumor_, by="VariantID.2") %>%
#   mutate(Tumor.VariantFreq=ifelse(is.na(Tumor.VariantFreq), 0, Tumor.VariantFreq),
#          in.tumor = factor(Tumor.VariantFreq > 0) #,
#          #Tumor.VariantType=ifelse(is.na(Tumor.VariantType), "Germline", Tumor.VariantType)
#   )   
## I don't think we need this last feature in the model, it is just for reference later, so maybe leave as NA


table(all.snv.plasma.McL$in.pbmc) # 39585 variants overlap PBMC + plasma ATLAS
#table(all.snv.plasma.McL$in.tumor) # 42340 variants overlap FFPE + plasma ATLAS . . . these are mostly germline mutations
dim(all.snv.plasma.McL)

# View all of the new columns we've added onto the plasma ATLAS dataset
names(all.snv.plasma.McL)[100:length(all.snv.plasma.McL)]

### 
# somatic Tumor.VariantType counts are low because most of the tumor variants are not detectable in the plasma;
# or Tumor.VariantType==NA.
#table(all.snv.plasma.McL$PatientID, all.snv.plasma.McL$Tumor.VariantType, useNA="always") 
# here we have about ~120 somatic variants in the plasma per patient
table(all.snv.plasma.McL$PatientID, all.snv.plasma.McL$VariantType, useNA="always")

table(all.snv.plasma.McL$VariantType) # about 2400 variants labeled as somatic/likely somatic/chip by the existing pipeline
sort(names(all.snv.plasma.McL))



FALSE  TRUE  <NA> 
 9800 45844     0 

[1] 55644   116

[1] "VariantType.old"         "finalKeep.old"          
 [3] "VariantID.2"             "in.dbSNP"               
 [5] "in.genome1000"           "in.cosmic"              
 [7] "SBS_Count"               "pTMB_norm.adj"          
 [9] "pTMB_norm.weighted"      "Fragment_Size_Bandwidth"
[11] "Fragment_Size_Mode"      "maxAF_TMB"              
[13] "tumorFraction"           "Plasma.VariantFreq"     
[15] "Germ.VariantFreq"        "Germ.VariantType"       
[17] "in.pbmc"

        
         CHIP mutations Germline Likely background Likely CHIP mutations
  170594              1     2390                47                     1
  170762              3     2201                62                     2
  171126              1     2077                75                     2
  171351              4     2295                60                     2
  171660              3     2234                60                     2
  172109              2     2178                61                     1
  172163              0     2263                61                     1
  172422              0     2166                52                     2
  172592              1     2146                32                     2
  172956              2     2240                62                     1
  173129              0     2109                51                     1
  173149              0     2100                67                     2
  173657              1     2090          


       CHIP mutations              Germline     Likely background 
                   33                 50450                  1216 
Likely CHIP mutations       Likely germline        Likely somatic 
                   36                  1101                   600 
              Somatic                  <NA> 
                 2208                     0 

[1] "AF"                         "AF.Baseline"               
  [3] "AF.diff"                    "AF.th"                     
  [5] "alt"                        "altDepth"                  
  [7] "altDepth.Baseline"          "altDepth.Baseline.filtered"
  [9] "altLOD"                     "Amino_acids"               
 [11] "avgEntropy"                 "bgFilter"                  
 [13] "BIOTYPE"                    "CANONICAL"                 
 [15] "CLIN_SIG"                   "Clinvar"                   
 [17] "Codons"                     "concordant"                
 [19] "Consequence"                "CopyNumber"                
 [21] "COSMIC"                     "COSMIC.CNT"                
 [23] "coVariant"                  "coVariantFilter"           
 [25] "dbSNP"                      "depth"                     
 [27] "DISTANCE"                   "DS.filter"                 
 [29] "DSCnt"                      "dscnt.Baseline.filtered"   
 [31] "end"                        "entropy"                   
 [33] "Existing_variation"         "EXON"                      
 [35] "externalSampleID"           "fc.ds.ratio"               
 [37] "Feature"                    "filtered_tlen_mean"        
 [39] "filtered_toend_numunique"   "filtered_total_base_count" 
 [41] "filteredCnt"                "filteredDSCnt"             
 [43] "filterPrevalence"           "filterScore"               
 [45] "FilterType"                 "filterVarType"             
 [47] "finalKeep"                  "finalKeep.old"             
 [49] "Fragment_Size_Bandwidth"    "Fragment_Size_Mode"        
 [51] "GC.Percent"                 "GeneID"                    
 [53] "generalFilter"              "genome1000"                
 [55] "Germ.VariantFreq"           "Germ.VariantType"          
 [57] "HGVSc"                      "HGVSp"                     
 [59] "highFrequent.inbatch"       "highQ.ratio"               
 [61] "ID"                         "IMPACT"                    
 [63] "in.cosmic"                  "in.dbSNP"                  
 [65] "in.genome1000"              "in.pbmc"                   
 [67] "INTRON"                     "localVariantFilter"        
 [69] "MAF.diff"                   "maxAF_TMB"                 
 [71] "mergedVariant"              "MNV"                       
 [73] "odds.ratio"                 "ol.coVariant"              
 [75] "PatientID"                  "PatientID.old"             
 [77] "Plasma.VariantFreq"         "prevalenceInternal"        
 [79] "pseudoDSCnt"                "pTMB_norm.adj"             
 [81] "pTMB_norm.weighted"         "pvalue"                    
 [83] "Qscore"                     "ReadEndFilter"             
 [85] "ref"                        "refDepth"                  
 [87] "repeatRegion"               "sampleDir"                 
 [89] "sampleFolder"               "sampleNames"               
 [91] "SBS_Count"                  "seqnames"                  
 [93] "sideVariant"                "sideVariant.AF.ratio"      
 [95] "snvmember"                  "SpecimenType"              
 [97] "start"                      "strand"                    
 [99] "SYMBOL"                     "tandemRepeat"              
[101] "tlen.ratio"                 "totalDepth"                
[103] "totalDepth.Baseline"        "trialVisitNum"             
[105] "tumorFraction"              "VARIANT_CLASS"             
[107] "VariantFreq"                "VariantID"                 
[109] "VariantID.2"                "VariantType"               
[111] "VariantType.old"            "white.list"                
[113] "width"                      "wl.tier"                   
[115] "X"                          "zScore.cnv"

In [79]:
# Ground truth = VariantType is somatic/CHIP after germline matched normal pipeline
# We are treating CHIP in same category as somatic here.

print("Raw labels")
table(all.snv.plasma.McL$VariantType.old) # raw pipeline labels

all.snv.plasma.McL <- all.snv.plasma.McL %>% 
  mutate(raw.label = factor(ifelse(grepl("somatic|chip", VariantType.old, ignore.case=T), "somatic","germline"), levels=c("somatic","germline")),
         matched.label=factor(ifelse(grepl("somatic|chip", VariantType, ignore.case=T), "somatic","germline"), levels=c("somatic","germline")),
         # truth.label = factor(ifelse(grepl("somatic|chip", Tumor.VariantType, ignore.case=T), "somatic","germline"), levels=c("somatic","germline"))
         # prevents weird treatment of NA values:
         truth.label = factor(ifelse(!is.na(Tumor.VariantType) & grepl("somatic|chip", Tumor.VariantType, ignore.case=T), "somatic","germline"), levels=c("somatic","germline")))

# Truth set will be: variantType = somatic | chip in FFPE (Tumor.VariantType column)

table(all.snv.plasma.McL$raw.label) 
table(all.snv.plasma.McL$matched.label)
table(all.snv.plasma.McL$truth.label)

# variantType = somatic | chip in FFPE
# match to plasma data set patientID


## wow, a very small subset of the plasma somatic variants are also called as somatic in the tumor
## maybe our pipeline somatic calling for tumor tissue isn't that good. might explore a less conservative truth label?
# all.snv.plasma.McL <- all.snv.plasma.McL %>% 
#                 mutate(truth.label.2 = factor(ifelse(matched.label=="somatic" & as.logical(in.tumor), "somatic", "germline"), levels=c("somatic","germline")))
# table(all.snv.plasma.McL$truth.label.2) # doesn't help much :-(
# somatic germline 
#      158    47939 


## this would be a good place to produce a venn diagram of how the three types of labels overlap for any variant with a somatic call
## some code for this in previous notebook


Plasma   <NA> 
 55644      0 


 CHIP mutations        Germline Likely germline  Likely somatic         Somatic 
             56           47018            1928            6629              13 
           <NA> 
              0 

ERROR: Error in eval(expr, envir, enclos): 
